# Calculating Biomass - Loop through raster

Derived from Gader, P. , 2020. “Calculate Vegetation Biomass from LiDAR Data in Python” https://www.neonscience.org/resources/learning-hub/tutorials/calc-biomass-py

In [138]:
#Import packages
import numpy as np
import os
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import sys
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import rasterio as rio
import glob
%matplotlib inline 

In [139]:
#Ignore warnings for loop
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#Import biomass specific libraries
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.measure import regionprops
from sklearn.ensemble import RandomForestRegressor
from skimage.segmentation import watershed

In [140]:
#Define the file of training data  
training_data_file = '/home/jovyan/NEON_CO2_Macrosystems_LIDAR/Code/Data/SJER_Biomass_Training.csv'

#Read in the training data from a CSV file
training_data = np.genfromtxt(training_data_file,delimiter=',') 

#Grab the biomass (Y) from the first line
biomass = training_data[:,0]

#Grab the biomass prdeictors from the remaining lines
biomass_predictors = training_data[:,1:12]

In [141]:
#Define paraemters for Random forest regressor
max_depth = 30

#Define regressor rules
regr_rf = RandomForestRegressor(max_depth=max_depth, random_state=2)

#Fit the biomass to regressor variables
regr_rf.fit(biomass_predictors,biomass)

RandomForestRegressor(max_depth=30, random_state=2)

In [142]:
#Further define regressor
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=2,
           verbose=0, warm_start=False)

RandomForestRegressor(max_depth=30, min_impurity_split=1e-07, n_estimators=10,
                      n_jobs=1, random_state=2)

In [78]:
#File for testing function
test = '/home/jovyan/NEON_CO2_Macrosystems_LIDAR/Code/Data/DP3.30015.001/2019_WREF_3/CanopyHeightModelGtif/NEON_D16_WREF_DP3_573000_5071000_CHM.tif'

In [143]:
#Function to open file and process 
def open_process(chm_file):
    chm_dataset = gdal.Open(chm_file)
    #Get the raster band object
    chm_raster = chm_dataset.GetRasterBand(1)
    #Get the NO DATA value
    noDataVal_chm = chm_raster.GetNoDataValue()
    #Get required metadata from CHM file
    cols_chm = chm_dataset.RasterXSize
    rows_chm = chm_dataset.RasterYSize
    bands_chm = chm_dataset.RasterCount
    mapinfo_chm =chm_dataset.GetGeoTransform()
    xMin = mapinfo_chm[0]
    yMax = mapinfo_chm[3]
    xMax = xMin + chm_dataset.RasterXSize/mapinfo_chm[1]
    yMin = yMax + chm_dataset.RasterYSize/mapinfo_chm[5]
    image_extent = (xMin,xMax,yMin,yMax)
    
    #Define array
    chm_array = chm_raster.ReadAsArray(0,0,cols_chm,rows_chm).astype(np.float)
   
    #Smooth the CHM using a gaussian filter to remove spurious points
    chm_array_smooth = ndi.gaussian_filter(chm_array,1,mode='constant',cval=0,truncate=2.0)
    chm_array_smooth[chm_array==0] = 0 
    
    #Calculate local maximum points in the smoothed CHM
    local_maxi = peak_local_max(chm_array_smooth,min_distance=10, threshold_abs=0, threshold_rel=0, indices=False)
    
    #Identify all the maximum points
    markers = ndi.label(local_maxi)[0]
    
    #Create a CHM mask so the segmentation will only occur on the trees
    chm_mask = chm_array_smooth
    chm_mask[chm_array_smooth != 0] = 1
    
    #Perfrom watershed segmentation        
    labels = watershed(chm_array_smooth, markers, mask=chm_mask)
    
    #Get the properties of each segment
    tree_properties = regionprops(labels,chm_array, ['Area','BoundingBox','Centroid','Orientation','MajorAxisLength','MinorAxisLength','MaxIntensity','MinIntensity'])

    #Determine how many individual trees were identified
    max_labels = labels.max()
    segment_labels = np.zeros(max_labels+1)
    segment_id = np.zeros(max_labels+1)

    for counter in range (1,max_labels+1):
        segment_labels[counter] = len(labels[labels==counter])
        segment_id[counter]=counter

    #Remove the non-zero elements
    segment_id = segment_id[np.nonzero(segment_labels)]
    
    #Change the labels to float
    labels = np.array((labels),dtype=float)
    #Change the zero labels to nans 
    labels[labels==0] = np.nan
    
        #Define several of the predictor variables
    area=np.zeros(len(tree_properties))
    diameter=np.zeros(len(tree_properties))
    max_tree_height=np.zeros(len(tree_properties))
    min_tree_height=np.zeros(len(tree_properties))
    
    #Retreive the predictor variables from the region properties
    for counter in range(0,len(tree_properties)):

        area[counter] = tree_properties[counter]['Area']
        diameter[counter] = tree_properties[counter]['MajorAxisLength']        
        max_tree_height[counter] =  tree_properties[counter]['MaxIntensity']    
        min_tree_height[counter] = tree_properties[counter]['MinIntensity'] 
    
    #Define the remaining predictor variables

    crown_geometric_volume_full=np.zeros(len(segment_id))
    crown_geometric_volume_50th_percentile=np.zeros(len(segment_id))
    crown_geometric_volume_60th_percentile=np.zeros(len(segment_id))
    crown_geometric_volume_70th_percentile=np.zeros(len(segment_id))
    percentile_50th=np.zeros(len(segment_id))
    percentile_60th=np.zeros(len(segment_id))
    percentile_70th=np.zeros(len(segment_id))
    
    #Cycle through all of the tree segments    
    counter=0
    for segment in segment_id:
        #Pull out the tree of interest
        indexes_of_tree = np.asarray(np.where(labels==segment)).T
        tree_data = chm_array[indexes_of_tree[:,0],indexes_of_tree[:,1]]
        #Calculate the geometric volume 
        crown_geometric_volume_full[counter]=np.sum([tree_data-np.min(tree_data)])

        #Pull out 50th percentile stats
        percentile_50th[counter]=np.percentile(tree_data,50)
        tree_data_50th = chm_array[indexes_of_tree[:,0],indexes_of_tree[:,1]]
        tree_data_50th[tree_data_50th>percentile_50th[counter]] = percentile_50th[counter]
        crown_geometric_volume_50th_percentile[counter]=np.sum([tree_data_50th-min_tree_height[counter]])

        #Pull out 60th percentile stats    
        percentile_60th[counter]=np.percentile(tree_data,60)
        tree_data_60th = chm_array[indexes_of_tree[:,0],indexes_of_tree[:,1]]
        tree_data_60th[tree_data_60th>percentile_60th[counter]] = percentile_60th[counter]
        crown_geometric_volume_60th_percentile[counter]=np.sum([tree_data_60th-min_tree_height[counter]])

        #Pull out 60th percentile stats 
        percentile_70th[counter]=np.percentile(tree_data,70)
        tree_data_70th = chm_array[indexes_of_tree[:,0],indexes_of_tree[:,1]]
        tree_data_70th[tree_data_70th>percentile_70th[counter]] = percentile_70th[counter]
        crown_geometric_volume_70th_percentile[counter]=np.sum([tree_data_70th-min_tree_height[counter]])

        counter=counter+1
        
        #Stack the predictor variables for all the individual trees
        all_training_data = np.stack([area,diameter,max_tree_height,min_tree_height,percentile_50th,percentile_60th,percentile_70th,crown_geometric_volume_full,
                              crown_geometric_volume_50th_percentile,crown_geometric_volume_60th_percentile,crown_geometric_volume_70th_percentile],axis=-1)
        
        #Apply the model to the predictive data
        all_training_data = np.nan_to_num(all_training_data) #set nan values
        pred_biomass = regr_rf.predict(all_training_data)
        
        #Set an out raster with the same size as the labels
        biomass_out = labels

        #Set counter to zero
        counter = 0
        #Assign each tree by the associated biomass
        for segment in segment_id:
            biomass_out[biomass_out==segment] = pred_biomass[counter]
            counter = counter+1
            
        #Get biomass stats for plotting
        mean_biomass = np.mean(pred_biomass)
        std_biomass = np.std(pred_biomass)
        min_biomass = np.min(pred_biomass)
        sum_biomass = np.sum(pred_biomass)

        #print('Sum of biomass is ',round(sum_biomass,2),' kg')     
        return sum_biomass

In [168]:
#Get list of files to loop through 
dirpath = '/home/jovyan/NEON_CO2_Macrosystems_LIDAR/Code/Data/DP3.30015.001/DP3.30015.001/2019/FullSite/D06/2019_KONZ_4/L3/DiscreteLidar/CanopyHeightModelGtif'
search_criteria = "*.tif"
q = os.path.join(dirpath, search_criteria)
dem_fps = glob.glob(q)

In [145]:
#Timer to keep track of loop
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [178]:
#Loop through each raster in each site and add biomass
biomass = 0
tqdm().pandas()
for i in tqdm(dem_fps):
    b = open_process(i)
    if b is None:
        b = 0
    biomass = biomass + b 

KeyboardInterrupt: 

In [173]:
#DO NOT RUN AGAIN
#MART_biomass = round(biomass/10**6,2)
print("The biomass at the WREF site is", MART_biomass)

The biomass at the WREF site is 2674.64


In [174]:
#DO NOT RUN AGAIN
#BONA_biomass = round(biomass/10**6,2)
print("The biomass at the BONA site is", BONA_biomass)

The biomass at the BONA site is 1787.57


In [175]:
#DO NOT RUN AGAIN
#KONZ_biomass = round(biomass/10**6,2)
print("The biomass at the KONZ site is", KONZ_biomass)

The biomass at the KONZ site is 670.55


In [177]:
#DO NOT RUN AGAIN
#NIWO_biomass = round(biomass/10**6,2)
print("The biomass at the NIWO site is", NIWO_biomass)

The biomass at the NIWO site is 935.53
